In [119]:
import xarray as xr
import hvplot.xarray
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tqdm import tqdm

## Look at different ocean data / simulation

In [8]:
fn = '/datadrive/CTD_data/axial_base_onlyprofilers.nc'
da = xr.open_dataarray(fn)

In [9]:
da.hvplot.image(x='dates', y='depth', rasterize=True)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


:DynamicMap   []
   :Image   [dates,depth]   (Axial Base SSP)

In [10]:
fn = '/datadrive/HYCOM_data/AxialSeamount_Caldera.zarr/'
hycom = xr.open_zarr(fn)

In [13]:
hycom.water_temp.hvplot.image(x='time', y='depth', rasterize=True)

:DynamicMap   []
   :Image   [time,depth]   (water_temp)

## Data Explorer (ungridded data)

In [2]:
# open dataset
fn = '/datadrive/CTD_data/data_explorer/ctd_data.nc'
ds = xr.open_dataset(fn)

In [3]:
# remove duplicates
duplicate_mask = np.diff(ds['time']) == np.array(0, dtype='timedelta64[ns]')
duplicate_mask = np.concatenate((np.array([False]), duplicate_mask))

ds_nodup = ds.isel({'row':~duplicate_mask})

In [90]:
ds_starttime = pd.Timestamp(ds_nodup['time'][0].values)
ds_endtime = pd.Timestamp(ds_nodup['time'][-1].values)

first_day = pd.Timestamp(ds_starttime.strftime('%Y-%m-%d'))

In [94]:
time_coord_day = pd.to_datetime(np.arange(first_day.value, ds_endtime.value, 3600*24*1e9,))

In [97]:
# slice dataset for single day
time_coord_full = ds_nodup.time.values

start_time = pd.Timestamp('2017-01-01')
end_time = pd.Timestamp('2017-01-02')

time_mask = (time_coord_full > start_time) & (time_coord_full < end_time)
time_ids = np.where(time_mask)[0]

ds_day = ds_nodup.isel({'row':time_ids})

In [13]:
for z in range(-100, 1):
    depth_mask = (ds_day.z == z).values
    
    break
    
np.where(depth_mask)

(array([], dtype=int64),)

In [100]:
var_names = list(ds_day.keys())
var_names.remove('time')
var_names.remove('z')

In [121]:
# Construct Xarray.DataArrays
das_grid = []
depth_coord = np.arange(0,-200, -1)
for var_name in var_names:
      das_grid.append(xr.DataArray(np.ones((len(time_coord_day), 200))*np.nan, dims=['time','depth'], coords={'depth':depth_coord, 'time':time_coord_day}, name=var_name))

ds_grid = xr.Dataset(dict(zip(var_names, das_grid)))

In [145]:
# loop through depth
for z in tqdm(depth_coord):
    depth_idxs = np.where((ds_day.z == z).values)[0]
    ds_single = ds_day.isel({'row':depth_idxs}).mean()
    
    for var_name in var_names:
        ds_grid[var_name].loc[{'depth':z, 'time':start_time}] = ds_single[var_name].values

100%|████████████████████████████████████████| 200/200 [00:01<00:00, 173.59it/s]
